In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from tqdm.notebook import tqdm
import numpy as np
from typing import NamedTuple
from itertools import chain

In [2]:
from policy_generator.policy_instances.envs.simple_arena import ActionSpace

In [3]:
#device to run model on 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = 'cpu'

In [4]:
class ObsSpace(NamedTuple):
    agent: np.ndarray
    agent_direction: int
    target: np.ndarray
    velocity: int

In [5]:
class PolicyNetwork(nn.Module):
    #Takes in observations and outputs actions
    def __init__(self, observation_space, action_space, shape):
        super(PolicyNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(observation_space, shape),
            nn.ReLU(),
            nn.Linear(shape, shape),
            nn.ReLU(),
            nn.Linear(shape, shape),
            nn.ReLU(),
            nn.Linear(shape, action_space)
        )
    
    #forward pass
    def forward(self, x):
        actions = self.model(x)
        action_probs = F.softmax(actions, dim=1)
        return action_probs

class PGBot:
    def __init__(self):
        self.model = None
        self.train_config = None
        
    def training_config(self, **kwargs):
        if kwargs:
            self.train_config = kwargs
        else:
            self.train_config = {
                'discount_factor': 0.96,
                'max_steps': 1000,
                'learning_rate': 1e-5,
                'num_episodes': 10000,
                'network_shape': 128,
                }
#             b_params = {'discount_factor': 0.9589431748343439,
#                         'learning_rate': 0.09167455769715335,
#                         'network_shape': 39}
            
    def _init_environment(self):
        #Make environment
        env = gym.make("policy_instances/SimpleArena-v0")

        #Init network
        network = PolicyNetwork(env.shape, env.action_space.n, self.train_config['network_shape']).to(DEVICE)

        #Init optimizer
        optimizer = optim.Adam(network.parameters(), lr=self.train_config['learning_rate'])
        return env, network, optimizer
    
    def train(self, verbose=0):
        env, network, optimizer = self._init_environment()
        
        #track scores
        scores = []

        #iterate through episodes
        for episode in tqdm(range(self.train_config['num_episodes'])):

            #reset environment, initiable variables
            state = env.reset()
            rewards = []
            log_probs = []
            score = 0
            actions_dist = []

            #generate episode
            for step in range(self.train_config['max_steps']):
                env.render()

                #select action
                action, lp = select_action(network, state)
                actions_dist.append(action)

                #execute action
                new_state, reward, done, _, _ = env.step(action)

                #track episode score
                score += reward

                #store reward and log probability
                rewards.append(reward)
                log_probs.append(lp)

                #end episode
                if done:
                    break

                #move into new state
                state = new_state

            #append score
            scores.append(score)
            if verbose > 0:
                print('Score:', score)

            #Calculate Gt (cumulative discounted rewards)
            discounted_rewards = []

            #track cumulative reward
            total_r = .0

            #iterate rewards from Gt to G0
            for r in reversed(rewards):
                # print(type(r), type(total_r), type(DISCOUNT_FACTOR), type(total_r ** DISCOUNT_FACTOR))
                # print(r, total_r, DISCOUNT_FACTOR, total_r ** DISCOUNT_FACTOR)

                #Base case: G(T) = r(T)
                #Recursive: G(t) = r(t) + G(t+1)^DISCOUNT
                if total_r < 0:
                    total_r = r - np.abs(total_r) ** self.train_config['discount_factor']
                else:
                    total_r = r + total_r ** self.train_config['discount_factor']

                #append to discounted rewards
                discounted_rewards.append(total_r)

            #reverse discounted rewards
            rewards = torch.tensor(discounted_rewards).to(DEVICE)
            rewards = torch.flip(rewards, [0])

            #adjusting policy parameters with gradient ascent
            loss = []
            for r, lp in zip(rewards, log_probs):
                #we add a negative sign since network will perform gradient descent and we are doing gradient ascent with REINFORCE
                loss.append(-r * lp)


            #Backpropagation
            optimizer.zero_grad()
            sum(loss).backward()
            # print('Loss:', sum(loss))
            if verbose > 0:
                print(list(zip(*np.unique(actions_dist, return_counts=True))))
            optimizer.step()

        env.close()
        return scores

            
def select_action(network, state):
    ''' Selects an action given current state
    Args:
    - network (Torch NN): network to process state
    - state (Array): Array of action space in an environment
    
    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = ObsSpace(**state[0] if isinstance(state, tuple) else state)
    unpack_state = list(chain(state.agent, state.target, [state.velocity, state.agent_direction]))
    state = torch.Tensor(unpack_state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    action_probs = network(state)
    
    #sample an action using the probability distribution
    m = Categorical(action_probs)
    action = m.sample()
    # print(m, action)
    
    #return action
    return action.item(), m.log_prob(action)

In [ ]:
bot = PGBot()
bot.training_config()
bot.train(1)

  0%|          | 0/10000 [00:00<?, ?it/s]

/home/beast/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/beast/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Score: -76
[(0, 3), (1, 11), (2, 11), (3, 47), (4, 28)]
Score: -46
[(0, 2), (1, 13), (2, 10), (3, 47), (4, 28)]
Score: -66
[(0, 3), (1, 15), (2, 7), (3, 33), (4, 42)]
Score: 158
[(0, 1), (1, 1), (2, 1), (3, 4), (4, 4)]
Score: -60
[(0, 4), (1, 11), (2, 14), (3, 42), (4, 29)]
Score: -62
[(0, 3), (1, 18), (2, 7), (3, 46), (4, 26)]
Score: -66
[(0, 4), (1, 15), (2, 9), (3, 37), (4, 35)]
Score: -67
[(0, 3), (1, 13), (2, 10), (3, 33), (4, 41)]
Score: -74
[(1, 14), (2, 5), (3, 42), (4, 39)]
Score: -58
[(0, 4), (1, 19), (2, 5), (3, 29), (4, 43)]
Score: -68
[(0, 1), (1, 11), (2, 3), (3, 48), (4, 37)]
Score: -46
[(0, 3), (1, 13), (2, 15), (3, 41), (4, 28)]
Score: -86
[(0, 5), (1, 10), (2, 5), (3, 51), (4, 29)]
Score: -56
[(0, 2), (1, 18), (2, 6), (3, 47), (4, 27)]
Score: -64
[(0, 2), (1, 15), (2, 10), (3, 40), (4, 33)]
Score: -50
[(0, 1), (1, 20), (2, 10), (3, 39), (4, 30)]
Score: -8
[(0, 7), (1, 20), (2, 3), (3, 45), (4, 25)]
Score: -68
[(0, 2), (1, 19), (2, 6), (3, 42), (4, 31)]
Score: -72
[(0,

Score: -58
[(0, 7), (1, 25), (2, 11), (3, 42), (4, 15)]
Score: -54
[(0, 6), (1, 32), (2, 7), (3, 39), (4, 16)]
Score: -42
[(0, 3), (1, 32), (2, 8), (3, 33), (4, 24)]
Score: -54
[(0, 8), (1, 26), (2, 9), (3, 34), (4, 23)]
Score: -52
[(0, 5), (1, 35), (2, 8), (3, 35), (4, 17)]
Score: -43
[(0, 4), (1, 27), (2, 11), (3, 30), (4, 28)]
Score: -50
[(0, 5), (1, 23), (2, 5), (3, 41), (4, 26)]
Score: -32
[(0, 8), (1, 34), (2, 7), (3, 27), (4, 24)]
Score: -44
[(0, 2), (1, 31), (2, 4), (3, 32), (4, 31)]
Score: -42
[(0, 7), (1, 33), (2, 11), (3, 31), (4, 18)]
Score: -66
[(0, 4), (1, 19), (2, 12), (3, 42), (4, 23)]
Score: -48
[(0, 8), (1, 27), (2, 5), (3, 36), (4, 24)]
Score: -38
[(0, 1), (1, 26), (2, 7), (3, 43), (4, 23)]
Score: -50
[(0, 6), (1, 26), (2, 11), (3, 41), (4, 16)]
Score: -52
[(0, 2), (1, 25), (2, 9), (3, 38), (4, 26)]
Score: -62
[(0, 5), (1, 29), (2, 13), (3, 23), (4, 30)]
Score: -56
[(0, 3), (1, 22), (2, 4), (3, 51), (4, 20)]
Score: -29
[(0, 3), (1, 36), (2, 10), (3, 30), (4, 21)]
Sco

In [7]:
import optuna

def objective(trial):
    bot = PGBot()
    bot.training_config(**{
        'discount_factor': trial.suggest_float('discount_factor', 0.90, 0.99),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'num_episodes': 100,
        'max_steps': 10000,
        'network_shape': trial.suggest_int('network_shape', 16, 128),
                    })
    score = bot.train()
    return -1 * np.mean(score)

study = optuna.create_study()
study.optimize(objective, n_trials=1000)

study.best_params

[I 2022-09-19 12:21:36,972] A new study created in memory with name: no-name-66e47096-aae4-4c0a-9082-e926e2d906e2


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-19 12:22:00,781] Trial 0 finished with value: 446.48 and parameters: {'discount_factor': 0.9873970997044688, 'learning_rate': 0.08556202411794087, 'network_shape': 125}. Best is trial 0 with value: 446.48.


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-19 12:22:24,002] Trial 1 finished with value: 446.11 and parameters: {'discount_factor': 0.9683505979911465, 'learning_rate': 0.08508350136598532, 'network_shape': 114}. Best is trial 1 with value: 446.11.


...

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-19 16:03:28,075] Trial 590 finished with value: 445.73 and parameters: {'discount_factor': 0.9618886736939993, 'learning_rate': 0.04646045343994011, 'network_shape': 30}. Best is trial 78 with value: 246.68.


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2022-09-19 16:03:36,422] Trial 591 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/home/beast/.local/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_828/3544524599.py", line 12, in objective
    score = bot.train()
  File "/tmp/ipykernel_828/1221072934.py", line 70, in train
    action, lp = select_action(network, state)
  File "/tmp/ipykernel_828/1221072934.py", line 158, in select_action
    m = Categorical(action_probs)
  File "/home/beast/.local/lib/python3.8/site-packages/torch/distributions/categorical.py", line 64, in __init__
    super(Categorical, self).__init__(batch_shape, validate_args=validate_args)
  File "/home/beast/.local/lib/python3.8/site-packages/torch/distributions/distribution.py", line 54, in __init__
    if not valid.all():
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
study.best_params

{'discount_factor': 0.9589431748343439,
 'learning_rate': 0.09167455769715335,
 'network_shape': 39}

In [ ]:
#discount factor for future utilities
DISCOUNT_FACTOR = 0.99

#number of episodes to run
NUM_EPISODES = 10000

#max steps per episode
MAX_STEPS = 1000

In [5]:
#Using a neural network to learn our policy parameters
class PolicyNetwork(nn.Module):
    
    #Takes in observations and outputs actions
    def __init__(self, observation_space, action_space):
        super(PolicyNetwork, self).__init__()
#         self.input_layer = 
#         self.relu = nn.ReLU(32, 32)
#         self.output_layer = nn.Linear(32, action_space)
        self.model = nn.Sequential(
            nn.Linear(observation_space, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_space)
        )
    
    #forward pass
    def forward(self, x):
        #input states
#         x = self.input_layer(x)
        
#         #relu activation
#         x = F.relu(x)
        
#         #actions
#         actions = self.output_layer(x)
        actions = self.model(x)
        
        #get softmax for a probability distribution
        action_probs = F.softmax(actions, dim=1)
        # print(action_probs)
        
        return action_probs

In [6]:
def select_action(network, state):
    ''' Selects an action given current state
    Args:
    - network (Torch NN): network to process state
    - state (Array): Array of action space in an environment
    
    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = ObsSpace(**state[0] if isinstance(state, tuple) else state)
    unpack_state = list(chain(state.agent, state.target, [state.velocity, state.agent_direction]))
    state = torch.Tensor(unpack_state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    action_probs = network(state)
    
    #sample an action using the probability distribution
    m = Categorical(action_probs)
    action = m.sample()
    # print(m, action)
    
    #return action
    return action.item(), m.log_prob(action)

In [7]:
#Make environment
env = gym.make("policy_instances/SimpleArena-v0")

#Init network
network = PolicyNetwork(env.shape, env.action_space.n).to(DEVICE)

#Init optimizer
optimizer = optim.Adam(network.parameters(), lr=1e-4)

In [8]:
state = env.reset()
select_action(network, state)

/home/beast/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(4, tensor([-0.7832], device='cuda:0', grad_fn=<SqueezeBackward1>))

In [9]:
#track scores
scores = []

#iterate through episodes
for episode in tqdm(range(NUM_EPISODES)):
    
    #reset environment, initiable variables
    state = env.reset()
    rewards = []
    log_probs = []
    score = 0
    actions_dist = []
    
    #generate episode
    for step in range(MAX_STEPS):
        env.render()
        
        #select action
        action, lp = select_action(network, state)
        actions_dist.append(action)
        
        #execute action
        new_state, reward, done, _, _ = env.step(action)
        
        #track episode score
        score += reward
        
        #store reward and log probability
        rewards.append(reward)
        log_probs.append(lp)
        
        #end episode
        if done:
            break
        
        #move into new state
        state = new_state
    
    #append score
    scores.append(score)
    print('Score:', score)
        
    #Calculate Gt (cumulative discounted rewards)
    discounted_rewards = []
    
    #track cumulative reward
    total_r = .0
    
    #iterate rewards from Gt to G0
    for r in reversed(rewards):
        # print(type(r), type(total_r), type(DISCOUNT_FACTOR), type(total_r ** DISCOUNT_FACTOR))
        # print(r, total_r, DISCOUNT_FACTOR, total_r ** DISCOUNT_FACTOR)
        
        #Base case: G(T) = r(T)
        #Recursive: G(t) = r(t) + G(t+1)^DISCOUNT
        if total_r < 0:
            total_r = r - np.abs(total_r) ** DISCOUNT_FACTOR
        else:
            total_r = r + total_r ** DISCOUNT_FACTOR
        
        #append to discounted rewards
        discounted_rewards.append(total_r)
    
    #reverse discounted rewards
    rewards = torch.tensor(discounted_rewards).to(DEVICE)
    rewards = torch.flip(rewards, [0])
    
    #adjusting policy parameters with gradient ascent
    loss = []
    for r, lp in zip(rewards, log_probs):
        #we add a negative sign since network will perform gradient descent and we are doing gradient ascent with REINFORCE
        loss.append(-r * lp)
    
        
    #Backpropagation
    optimizer.zero_grad()
    sum(loss).backward()
    # print('Loss:', sum(loss))
    print(list(zip(*np.unique(actions_dist, return_counts=True))))
    optimizer.step()

env.close()

  0%|          | 0/10000 [00:00<?, ?it/s]

/home/beast/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Score: 142
[(1, 1), (3, 1), (4, 4)]
Score: -352
[(0, 4), (1, 16), (2, 14), (3, 34), (4, 33)]
Score: -379
[(0, 8), (1, 16), (2, 16), (3, 24), (4, 37)]
Score: 77
[(1, 3), (2, 1), (3, 6), (4, 26)]
Score: -398
[(0, 6), (1, 9), (2, 12), (3, 21), (4, 53)]
Score: -303
[(0, 7), (1, 22), (2, 11), (3, 21), (4, 40)]
Score: -386
[(0, 2), (1, 19), (2, 5), (3, 17), (4, 58)]
Score: -408
[(0, 6), (1, 8), (2, 5), (3, 29), (4, 53)]
Score: -374
[(0, 3), (1, 16), (2, 12), (3, 17), (4, 53)]
Score: -379
[(0, 7), (1, 18), (2, 10), (3, 13), (4, 53)]
Score: -402
[(0, 3), (1, 6), (2, 2), (3, 24), (4, 66)]
Score: -396
[(0, 5), (1, 12), (2, 2), (3, 14), (4, 68)]
Score: -359
[(0, 6), (1, 16), (2, 6), (3, 23), (4, 50)]
Score: -401
[(0, 4), (1, 13), (2, 7), (3, 23), (4, 54)]
Score: -381
[(0, 3), (1, 17), (2, 14), (3, 21), (4, 46)]
Score: -369
[(0, 4), (1, 13), (2, 8), (3, 22), (4, 54)]
Score: -385
[(0, 2), (1, 15), (2, 1), (3, 23), (4, 60)]
Score: -385
[(0, 5), (1, 12), (2, 10), (3, 17), (4, 57)]
Score: -429
[(0, 2)

Score: -311
[(0, 19), (1, 51), (2, 15), (3, 4), (4, 12)]
Score: -292
[(0, 19), (1, 49), (2, 7), (3, 11), (4, 15)]
Score: 52
[(0, 3), (1, 29), (2, 9), (3, 7), (4, 10)]
Score: -277
[(0, 14), (1, 44), (2, 18), (3, 9), (4, 16)]
Score: -501
[(0, 11), (1, 47), (2, 17), (3, 10), (4, 16)]
Score: -301
[(0, 14), (1, 54), (2, 7), (3, 9), (4, 17)]
Score: -317
[(0, 10), (1, 39), (2, 19), (3, 15), (4, 18)]
Score: -311
[(0, 11), (1, 50), (2, 12), (3, 16), (4, 12)]
Score: -329
[(0, 13), (1, 39), (2, 20), (3, 11), (4, 18)]
Score: -298
[(0, 11), (1, 44), (2, 19), (3, 11), (4, 16)]
Score: -25
[(0, 9), (1, 49), (2, 11), (3, 11), (4, 15)]
Score: -309
[(0, 13), (1, 48), (2, 12), (3, 14), (4, 14)]
Score: -341
[(0, 8), (1, 28), (2, 26), (3, 20), (4, 19)]
Score: -325
[(0, 11), (1, 51), (2, 17), (3, 10), (4, 12)]
Score: -323
[(0, 2), (1, 42), (2, 24), (3, 12), (4, 21)]
Score: -318
[(0, 16), (1, 54), (2, 17), (3, 6), (4, 8)]
Score: 11
[(0, 6), (1, 32), (2, 17), (3, 7), (4, 11)]
Score: -311
[(0, 16), (1, 43), (2,

Score: -322
[(0, 21), (1, 25), (2, 20), (3, 10), (4, 25)]
Score: -313
[(0, 28), (1, 25), (2, 13), (3, 12), (4, 23)]
Score: -284
[(0, 35), (1, 25), (2, 11), (3, 8), (4, 22)]
Score: -314
[(0, 27), (1, 25), (2, 13), (3, 9), (4, 27)]
Score: -303
[(0, 24), (1, 41), (2, 12), (3, 8), (4, 16)]
Score: 140
[(0, 8), (1, 7), (2, 1), (3, 1), (4, 1)]
Score: -308
[(0, 26), (1, 30), (2, 17), (3, 11), (4, 17)]
Score: -307
[(0, 20), (1, 32), (2, 12), (3, 11), (4, 26)]
Score: -301
[(0, 19), (1, 30), (2, 16), (3, 14), (4, 22)]
Score: -304
[(0, 24), (1, 39), (2, 14), (3, 11), (4, 13)]
Score: -293
[(0, 27), (1, 36), (2, 13), (3, 10), (4, 15)]
Score: -265
[(0, 32), (1, 27), (2, 13), (3, 13), (4, 16)]
Score: -265
[(0, 21), (1, 48), (2, 10), (3, 8), (4, 14)]
Score: -293
[(0, 30), (1, 38), (2, 17), (3, 10), (4, 6)]
Score: -284
[(0, 25), (1, 37), (2, 15), (3, 10), (4, 14)]
Score: -314
[(0, 37), (1, 26), (2, 14), (3, 10), (4, 14)]
Score: -326
[(0, 26), (1, 29), (2, 22), (3, 11), (4, 13)]
Score: -278
[(0, 44), (1,

...

Score: -258
[(0, 45), (1, 35), (2, 10), (3, 6), (4, 5)]
Score: -323
[(0, 41), (1, 30), (2, 17), (3, 3), (4, 10)]
Score: -245
[(0, 48), (1, 35), (2, 7), (3, 3), (4, 8)]
Score: -261
[(0, 44), (1, 39), (2, 13), (3, 2), (4, 3)]
Score: -268
[(0, 47), (1, 33), (2, 10), (3, 4), (4, 7)]
Score: -276
[(0, 53), (1, 30), (2, 8), (3, 6), (4, 4)]
Score: -288
[(0, 37), (1, 31), (2, 10), (3, 9), (4, 14)]
Score: -264
[(0, 51), (1, 33), (2, 2), (3, 10), (4, 5)]
Score: -283
[(0, 47), (1, 22), (2, 17), (3, 10), (4, 5)]
Score: -246
[(0, 42), (1, 29), (2, 15), (3, 9), (4, 6)]
Score: -253
[(0, 55), (1, 30), (2, 8), (3, 3), (4, 5)]
Score: -239
[(0, 48), (1, 37), (2, 4), (3, 8), (4, 4)]
Score: -281
[(0, 49), (1, 26), (2, 11), (3, 6), (4, 9)]
Score: -248
[(0, 52), (1, 34), (2, 9), (3, 4), (4, 2)]
Score: -256
[(0, 43), (1, 34), (2, 8), (3, 8), (4, 8)]
Score: -288
[(0, 50), (1, 37), (2, 5), (3, 3), (4, 6)]
Score: -272
[(0, 49), (1, 30), (2, 9), (3, 5), (4, 8)]
Score: -280
[(0, 40), (1, 34), (2, 13), (3, 9), (4, 5

Score: -278
[(0, 50), (1, 33), (2, 6), (3, 6), (4, 6)]
Score: -249
[(0, 50), (1, 31), (2, 12), (3, 4), (4, 4)]
Score: -268
[(0, 41), (1, 41), (2, 10), (3, 5), (4, 4)]
Score: -294
[(0, 52), (1, 29), (2, 10), (3, 3), (4, 7)]
Score: -269
[(0, 49), (1, 30), (2, 9), (3, 9), (4, 4)]
Score: -263
[(0, 46), (1, 26), (2, 12), (3, 8), (4, 9)]
Score: -234
[(0, 49), (1, 34), (2, 8), (3, 2), (4, 8)]
Score: -273
[(0, 50), (1, 26), (2, 13), (3, 7), (4, 5)]
Score: -272
[(0, 41), (1, 28), (2, 13), (3, 9), (4, 10)]
Score: 29
[(0, 35), (1, 18), (2, 11), (3, 2), (4, 7)]
Score: -303
[(0, 45), (1, 23), (2, 16), (3, 6), (4, 11)]
Score: -242
[(0, 41), (1, 30), (2, 11), (3, 11), (4, 8)]
Score: -301
[(0, 47), (1, 22), (2, 10), (3, 11), (4, 11)]
Score: -239
[(0, 47), (1, 34), (2, 8), (3, 6), (4, 6)]
Score: -241
[(0, 53), (1, 28), (2, 10), (3, 6), (4, 4)]
Score: -306
[(0, 44), (1, 23), (2, 19), (3, 10), (4, 5)]
Score: -300
[(0, 56), (1, 21), (2, 11), (3, 9), (4, 4)]
Score: -269
[(0, 51), (1, 34), (2, 11), (3, 3), 

Score: -292
[(0, 43), (1, 21), (2, 15), (3, 5), (4, 17)]
Score: -272
[(0, 50), (1, 15), (2, 18), (3, 13), (4, 5)]
Score: -323
[(0, 54), (1, 14), (2, 19), (3, 5), (4, 9)]
Score: -253
[(0, 48), (1, 25), (2, 14), (3, 7), (4, 7)]
Score: -317
[(0, 49), (1, 18), (2, 15), (3, 10), (4, 9)]
Score: -282
[(0, 60), (1, 13), (2, 12), (3, 9), (4, 7)]
Score: -285
[(0, 60), (1, 16), (2, 13), (3, 6), (4, 6)]
Score: -302
[(0, 48), (1, 19), (2, 14), (3, 12), (4, 8)]
Score: -289
[(0, 64), (1, 15), (2, 9), (3, 8), (4, 5)]
Score: -235
[(0, 52), (1, 19), (2, 15), (3, 7), (4, 8)]
Score: -243
[(0, 57), (1, 22), (2, 6), (3, 13), (4, 3)]
Score: -272
[(0, 58), (1, 12), (2, 10), (3, 8), (4, 13)]
Score: -294
[(0, 60), (1, 8), (2, 16), (3, 11), (4, 6)]
Score: -275
[(0, 57), (1, 16), (2, 15), (3, 6), (4, 7)]
Score: -280
[(0, 59), (1, 14), (2, 15), (3, 7), (4, 6)]
Score: -299
[(0, 61), (1, 11), (2, 13), (3, 10), (4, 6)]
Score: -293
[(0, 56), (1, 15), (2, 8), (3, 12), (4, 10)]
Score: -255
[(0, 58), (1, 23), (2, 9), (3,

Score: -289
[(0, 58), (1, 11), (2, 8), (3, 10), (4, 14)]
Score: 34
[(0, 17), (1, 6), (2, 12), (3, 8), (4, 12)]
Score: -307
[(0, 46), (1, 19), (2, 12), (3, 12), (4, 12)]
Score: -311
[(0, 45), (1, 18), (2, 19), (3, 11), (4, 8)]
Score: -290
[(0, 57), (1, 13), (2, 14), (3, 12), (4, 5)]
Score: -299
[(0, 58), (1, 12), (2, 16), (3, 7), (4, 8)]
Score: -321
[(0, 52), (1, 15), (2, 15), (3, 10), (4, 9)]
Score: -292
[(0, 38), (1, 18), (2, 15), (3, 14), (4, 16)]
Score: -337
[(0, 43), (1, 18), (2, 15), (3, 11), (4, 14)]
Score: -274
[(0, 57), (1, 13), (2, 8), (3, 11), (4, 12)]
Score: -292
[(0, 57), (1, 17), (2, 12), (3, 7), (4, 8)]
Score: -280
[(0, 55), (1, 18), (2, 10), (3, 7), (4, 11)]
Score: -319
[(0, 55), (1, 12), (2, 14), (3, 7), (4, 13)]
Score: -294
[(0, 56), (1, 11), (2, 14), (3, 13), (4, 7)]
Score: -286
[(0, 49), (1, 15), (2, 16), (3, 13), (4, 8)]
Score: -252
[(0, 50), (1, 14), (2, 11), (3, 10), (4, 16)]
Score: -246
[(0, 71), (1, 11), (2, 7), (3, 9), (4, 3)]
Score: -280
[(0, 57), (1, 15), (2,

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np

sns.set()

plt.plot(scores)
plt.ylabel('score')
plt.xlabel('episodes')
plt.title('Training score of SimpleArena with REINFORCE')

reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1, 1), np.array(scores).reshape(-1, 1))
y_pred = reg.predict(np.arange(len(scores)).reshape(-1, 1))
plt.plot(y_pred)
plt.show()

In [ ]:
done = False
state = env.reset()
scores = []

for _ in range(10):
    state = env.reset()
    done = False
    score = 0
    while not done:
        # env.render()
        action = env.action_space.sample()
        new_state, reward, done, info, _ = env.step(action)
        score += reward
        state = new_state
    scores.append(score)
env.close()

In [ ]:
done = False
state = env.reset()
scores = []

for _ in tqdm(range(50)):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action, lp = select_action(network, state)
        new_state, reward, done, info, _ = env.step(action)
        print(state, action)
        score += reward
        state = new_state
    scores.append(score)
env.close()

In [ ]:
np.array(scores).mean()

In [ ]:
done = False
state = env.reset()
scores = []

for _ in tqdm(range(50)):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action, lp = select_action(network, state)
        new_state, reward, done, info, _ = env.step(action)
        print(state, action)
        score += reward
        state = new_state
    scores.append(score)
env.close()